# Warning!

The current (Sep 2017) implementation of Hermite elements for $d \ge 2$ is bogus (see [Poisson2D](Poisson2D.ipynb) too): The Hermite trafo is applied to the dof at the barycenter, which is just an evaluation dof. This means that we try to access derivatives at this node which don't exist, resulting in access beyond the end of the array of basis values at integration points in the generated C++ code in `tabulate_tensor()`. We need to modify the intermediate representation of the terms in the integral to account for this dof, by looping first through the ones at the vertices then considering the last one without multiplication by the $H$ matrix, etc. An example value for `trans_integrals` for a form `v*dx` follows:

In [ ]:
trans_integrals = [(6, {(('j', 0, 9),): [[{'H', 'det'}, {6}, {'FE0'}, set(), {}],
                             [('j', '((FE0[ip][3*(j/3)+0])*(H[3 * (j % 3) + 0]) + ' +
                                    '(FE0[ip][3*(j/3)+1])*(H[3 * (j % 3) + 1]) + ' +
                                    '(FE0[ip][3*(j/3)+2])*(H[3 * (j % 3) + 2]))*W6[ip]*det', 7)]]},
                   {}, {}, None, {}),
                  (6, {(('j', 9, 10),): [[{'H', 'det'}, {6}, {'FE0'}, set(), {}],
                             [('j', '(FE0[ip][3*(j/3)+0])*W6[ip]*det', 7)]]},
                   {}, {}, None, {})]
#points, terms, functions, ip_consts, coordinate, conditionals = trans_integrals[1]

In [ ]:
from dolfin import *

parameters['form_compiler']['representation'] = 'quadrature'
# Optimization options for the form compiler
# MBD: Disable these until I've finished optimizedquadraturetransformer.py
parameters["form_compiler"]["cpp_optimize"] = False
parameters["form_compiler"]["optimize"] = False


# Make mesh ghosted for evaluation of DG terms
parameters["ghost_mode"] = "shared_facet"
def domain(nx, ny):
    return RectangleMesh(Point(0,-pi/2), Point(pi, pi/2), nx, ny, 'crossed')

W = FunctionSpace(domain(10,10), "Hermite", 3)
p = TrialFunction(W)
q = TestFunction(W)
a = inner(nabla_grad(p), nabla_grad(q))*dx
L = 10*q*dx
u = Function(W)
solve(a==L, u)
#b = assemble(L)
plot(u)

# Setting

This notebook solves the Biharmonic equation,

$$\nabla^4 u(x, y) = f(x, y)$$

on the square $[0, \pi] \times [-\pi/2,\pi/2]$ with source f given by

$$f(x, y) = 4 \sin(x) \cos(y)$$

and boundary conditions given by

$$u(x, y)         = 0$$
$$\nabla^2 u(x, y) = 0$$

using a discontinuous Galerkin formulation (interior penalty method). The analytic solution is

$$u(x,y) = \sin(x) \cos(y).$$

**TODO:** I should check whether the formulation makes sense for Hermite elements and what guarantees they provide.

# Implementation

In [ ]:
from dolfin import *
%matplotlib inline
import nbimporter
from boundary import apply_dirichlet_hermite
from dofs import list_hermite_dofs
from utils import ExpressionAD, Msg
from interpolation import interpolate_hermite
import matplotlib.pyplot as pl
import numpy as np

parameters['form_compiler']['representation'] = 'quadrature'
# Optimization options for the form compiler
# MBD: Disable these until I've finished optimizedquadraturetransformer.py
parameters["form_compiler"]["cpp_optimize"] = False
parameters["form_compiler"]["optimize"] = False


# Make mesh ghosted for evaluation of DG terms
parameters["ghost_mode"] = "shared_facet"


def sol(x, y):
    """ Analytic solution. """
    import autograd.numpy as np
    return np.sin(x)*np.cos(y)

def biharmonic(V:FunctionSpace) -> Function:
    """ Solves the biharmonic equation with a non-conforming discretisation.
    Accepts either Lagrange or Hermite FunctionSpaces.

    This code is based on DOLFIN's homonymous demo (C) 2009 Kristian B. Oelgaard
    """
    class DirichletBoundary(SubDomain):
        def inside(self, x, on_boundary):
            return on_boundary

    u0 = project(Constant(0.0), V)  # MBD: Need this for Hermite (interpolation doesn't work)
    bc = DirichletBC(V, u0, DirichletBoundary())

    u = TrialFunction(V)
    v = TestFunction(V)

    # Define normal component, mesh size and right-hand side
    h = CellSize(V.mesh())
    h_avg = (h('+') + h('-'))/2.0
    n = FacetNormal(V.mesh())
    
    def source(x,y):
        #import autograd.numpy as np
        #return 4.0*np.pi**4*np.sin(np.pi*x)*np.sin(np.pi*y)
        return 4*sol(x,y)
    f = ExpressionAD(fun=source, degree=3)

    # Penalty parameter
    alpha = Constant(6.0)

    # Define bilinear form
    a = inner(div(grad(u)), div(grad(v)))*dx \
        - inner(avg(div(grad(u))), jump(grad(v), n))*dS \
        - inner(jump(grad(u), n), avg(div(grad(v))))*dS \
        + alpha/h_avg*inner(jump(grad(u),n), jump(grad(v),n))*dS
    
    # Solve variational problem
    u = Function(V)
    if V.ufl_element().family().lower() == 'hermite':
        #L = interpolate_hermite(f, V)*v*dx  # nodal interpolation is much worse than projection
        with Msg("Assembling"):
            L = project(f, V)*v*dx
            A = assemble(a)
            b = assemble(L)
        with Msg("Applying Hermite BCs"):
            apply_dirichlet_hermite(A, b, bc)
        with Msg("Solving"):
            solve(A, u.vector(), b)
    else:
        #%debug -b /home/fenics/local/lib/python2.7/site-packages/ffc/quadrature/quadraturetransformerbase.py:347 solve(a == L, u, bc)
        L = f*v*dx
        with Msg("Solving"):
            solve(a == L, u, bc)
    
    return u

# Results

In [ ]:
def domain(nx, ny):
    return RectangleMesh(Point(0,-pi/2), Point(pi, pi/2), nx, ny, 'crossed')

In [ ]:
W = FunctionSpace(domain(10,10), "Lagrange", 3)
solution = project(ExpressionAD(fun=sol, degree=3), W)
lag = biharmonic(W)

In [ ]:
V = FunctionSpace(W.mesh(), "Hermite", 3)
her = biharmonic(V)

In [ ]:
pl.figure(figsize=(14,4))
pl.subplot(1,3,1)
plot(lag, title="Lagrange elements", cmap='hot')
pl.subplot(1,3,2)
plot(her, title="Hermite elements", cmap='hot')
pl.subplot(1,3,3)
plot(solution, title="Analytic solution", cmap='hot')
#pl.savefig("img/biharmonic-solutions.eps")

We compute now the pointwise relative difference, defined as the quotient 

$$\frac{u_h - u_l}{||u||_\infty}.$$

In [ ]:
# We evaluate the solutions only at the vertices of the mesh to avoid any weird effects
cc = W.mesh().coordinates()
xx = np.array(sorted(list(set(cc[:,0]))))
yy = np.array(sorted(list(set(cc[:,1]))))

infnorm = sol(xx,yy).max()

pl.figure(figsize=(16,6))
pl.subplot(1,2,1)
for y in yy[::7]:
    pl.plot(xx, [np.abs((lag(x,y) - sol(x,y)))/infnorm for x in xx],
            label='@%.1f' % y)
pl.title("Relative error at several ordinates (Lagrange)")
_ = pl.legend()
pl.subplot(1,2,2)
for y in yy[::7]:
    pl.plot(xx, [np.abs((her(x,y) - sol(x,y)))/infnorm for x in xx],
            label='@%.1f' % y)
pl.title("Relative error at several ordinates (Hermite)")
_ = pl.legend()

pl.savefig("img/biharmonic-approx-error.eps")

The difference between both approximations is strange:

In [ ]:
pl.figure(figsize=(12,8))
for y in yy[::7]:
    pl.plot(xx, [abs(lag(x, y) - her(x, y)) for x in xx], label='@%.1f' % y)
pl.title("$u_h - u_l$")
_ = pl.legend()

The maximal difference is below 0.1%:

In [ ]:
def relative_error(lag, her):
    lv = lag.vector().array()
    hv = her.vector().array()
    lagrange_dofs = list(set(range(V.dim())) - set(list_hermite_dofs(V)))
    lmin, lmax, hmin, hmax = lv.min(), lv.max(), hv[lagrange_dofs].min(), hv[lagrange_dofs].max()
    return 100*abs(hmax - lmax)/lmax

relative_error(lag, her)

In [ ]:
errors = []
numdofs_her = []
numdofs_lag = []
times_lag = []
times_her = []
sizes = range(5, 35, 5)
dolfin.DEBUG = Msg.output_level - 1  # Disable Msg's default output
for s in sizes:
    W = FunctionSpace(domain(s,s), "Lagrange", 3)
    with Msg("Computing Lagrange solution with s=%d" % s, level=0):
        lag = biharmonic(W)
    times_lag.append(Msg.last)
    numdofs_lag.append(W.dim())
    
    V = FunctionSpace(W.mesh(), "Hermite", 3)
    with Msg("Computing Hermite solution with s=%d" % s, level=0):
        her = biharmonic(V)
    times_her.append(Msg.last)
    numdofs_her.append(V.dim())

    errors.append(relative_error(lag, her))

numdofs_lag = np.array(numdofs_lag)
numdofs_her = np.array(numdofs_her)
times_lag = np.array(times_lag)
times_her = np.array(times_her)
errors = np.array(errors)

Degree 3, sizes: 10,15,20,25,30,35
```python
>>> errors, numdofs_lag, numdofs_her

   (array([ 0.63554695,  0.44286782,  0.35878701,  0.29271231,  0.25026288,
         0.21675917]),
    array([ 1861,  4141,  7321, 11401, 16381, 22261]),
    array([ 1063,  2343,  4123,  6403,  9183, 12463]))
```
and 
```python
# Manually copied
times_lag = [0.221, 0.453, 0.800, 1.253, 1.809, 2.483]
times_her = [0.858, 12.935, 70.857, 240.694, 1012.162, 3414.528]
```

In [ ]:
pl.figure(figsize=(10,5))
pl.plot(sizes, errors/100, label='diff')
_ = pl.title("Relative maximal difference")
#pl.savefig("img/biharmonic-diffs.eps")

In [ ]:
pl.figure(figsize=(20,8))
pl.subplot(1,2,1)
pl.tight_layout(w_pad=2.5, rect=(0.05, 0.05, 0.95, 0.95))
pl.plot(numdofs_her, times_her, label='her')
pl.ylabel('seconds')
pl.xlabel('dofs')
_ = pl.title("Hermite")
pl.subplot(1,2,2)
pl.plot(numdofs_lag, times_lag, label='lag')
pl.xlabel('dofs')
_ = pl.title("Lagrange")
#pl.savefig("img/biharmonic-times.eps")

In [ ]:
(max(times_lag) - min(times_lag)) / (numdofs_lag.max() - numdofs_lag.min())

The following plot is weird, but this is likely to be because of dolfin being confused by Hermite dofs:

In [ ]:
z = project(lag-her, V)
print(norm(z))
_ = plot(z, title="Difference", cmap='bone')

Save the solutions to files:

In [ ]:
File("biharmonic-hermite.pvd") << her
File("biharmonic-lagrange.pvd") << lag